In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import HistGradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/3-c-crab-age-prediction/sample_submission.csv
/kaggle/input/3-c-crab-age-prediction/train.csv
/kaggle/input/3-c-crab-age-prediction/test.csv


In [2]:
df = pd.read_csv("/kaggle/input/3-c-crab-age-prediction/train.csv")
df["Volume"] = df["Length"] * df["Diameter"] * df["Height"]
df["MeatRatio"] = df["Shucked Weight"] / df["Weight"]
df["ShellRatio"] = df["Shell Weight"] / df["Weight"]
X = df.drop(columns=["Age", "id"])
y = df["Age"]

categorical = ["Sex"]
numeric = [col for col in X.columns if col != "Sex"]
preprocessor = ColumnTransformer([
    ("cat", OneHotEncoder(handle_unknown="ignore"), categorical),
    ("num", "passthrough", numeric)
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [3]:
sk_model = Pipeline([
    ("prep", preprocessor),
    ("reg", HistGradientBoostingRegressor(random_state=42))
])
sk_model.fit(X_train, y_train)
y_pred_sk = sk_model.predict(X_test)
mae_sk = mean_absolute_error(y_test, y_pred_sk)
print("Sklearn MAE:", mae_sk)

test_df = pd.read_csv("/kaggle/input/3-c-crab-age-prediction/test.csv")
test_df["Volume"] = test_df["Length"] * test_df["Diameter"] * test_df["Height"]
test_df["MeatRatio"] = test_df["Shucked Weight"] / test_df["Weight"]
test_df["ShellRatio"] = test_df["Shell Weight"] / test_df["Weight"]
test_df["MeatRatio"] = np.where(test_df["Weight"] == 0, 0, test_df["MeatRatio"])
test_df["ShellRatio"] = np.where(test_df["Weight"] == 0, 0, test_df["ShellRatio"])
ids = test_df["id"]
X_final_test = test_df.drop(columns=["id"])

predictions = sk_model.predict(X_final_test)
submission = pd.DataFrame({
    "id": ids,
    "Age": predictions
})

#submission.to_csv("submission.csv", index=False)

Sklearn MAE: 1.3448091287598107


In [4]:
xgb_model = Pipeline([
    ("prep", preprocessor),
    ("reg", XGBRegressor(objective="reg:squarederror", random_state=42))
])
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
print("XGBoost MAE:", mae_xgb)


XGBoost MAE: 1.4076400109926859


In [5]:
estimators = [
    ("xgb", XGBRegressor(objective="reg:squarederror", random_state=42)),
    ("hgb", HistGradientBoostingRegressor(random_state=42))
]
stack_model = Pipeline([
    ("prep", preprocessor),
    ("stack", StackingRegressor(estimators=estimators, final_estimator=Ridge()))
])
stack_model.fit(X_train, y_train)
y_pred_stack = stack_model.predict(X_test)
mae_stack = mean_absolute_error(y_test, y_pred_stack)
print("Stacked MAE:", mae_stack)


Stacked MAE: 1.345755036464993
